In [1]:
#Lo primero es las librerias
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
import pandas as pd
import numpy as np
import math
import git
import datetime
import warnings
warnings.filterwarnings("ignore")


def recuperarData():
    link ='https://github.com/Sud-Austral/Datos_Panama/blob/master/datacovidpa/Localiza%20PN%20v1.xlsx?raw=true'
    data = pd.read_excel(link,'TODO CORREGIMIENTO')
    data.drop([0], inplace=True)
    data.columns = data.iloc[0]
    data.drop([1], inplace=True)
    our_data = data[['CD_Corr', 'Corregimiento', 'Fecha', 'Nuevos Confirmados', 'Nuevos Fallecidos', 'Nuevos Recuperados', 'Población']]
    now = datetime.datetime.now()
    our_data_filter = our_data[our_data['Fecha'] < now]
    our_data_filter.rename(columns={'Nuevos Confirmados': 'NuevosConfirmados', 'Nuevos Fallecidos': 'NuevosFallecidos',
                         'Nuevos Recuperados': 'NuevosRecuperados', 'Población': 'Poblacion'}, inplace=True)

    our_data_filter['NuevosConfirmados'] = pd.to_numeric(our_data_filter['NuevosConfirmados'])
    our_data_filter['NuevosFallecidos'] = pd.to_numeric(our_data_filter['NuevosFallecidos'])
    our_data_filter['NuevosRecuperados'] = pd.to_numeric(our_data_filter['NuevosRecuperados'])
    our_data_filter['Poblacion'] = pd.to_numeric(our_data_filter['Poblacion'])
    return our_data_filter

def limpiarData(dataframe):
    # eliminamos filas que no importan y asignamos la primera como cabecera
    dt = dataframe
    return our_data

def preCalculo(dt):
    our_data_filter = dt.iloc[1:]
    our_data_filter['NuevosConfirmados'] = our_data_filter['NuevosConfirmados'] + our_data_filter['NuevosFallecidos']
    our_data_filter['NuevosConfirmadosacum'] = our_data_filter['NuevosConfirmados'].cumsum()
    our_data_filter['NuevosRecuperadosacum'] = our_data_filter['NuevosRecuperados'].cumsum()
    our_data_filter["Susceptibles"] = our_data_filter['Poblacion']-our_data_filter['NuevosConfirmadosacum']-our_data_filter['NuevosRecuperadosacum']
    our_data_filter["pob_log_Susceptibles"] = our_data_filter['Poblacion']*np.log(our_data_filter["Susceptibles"])
    our_data_filter_un_final = our_data_filter[our_data_filter['NuevosRecuperadosacum'] > 0]  
    if(our_data_filter_un_final.shape[0] > 0):
        return our_data_filter_un_final
    else:
        dt["NuevosConfirmadosacum"] = 0
        dt["NuevosRecuperadosacum"] = 0
        dt["Susceptibles"] = 0
        dt["pob_log_Susceptibles"] = 0
        return dt.tail(1)

def calculo(dt):
    our_data_filter_un_final = dt
    y = our_data_filter_un_final["pob_log_Susceptibles"]
    lista_R0 = []
    for i in range(len(y)):
        y = our_data_filter_un_final["pob_log_Susceptibles"]
        x = our_data_filter_un_final["NuevosRecuperadosacum"]
        yy = y.head(i+1)
        xx = x.head(i+1)
    
        # Creamos el objeto de Regresión Lineal
        regresion_lineal = linear_model.LinearRegression()
    
        regresion_lineal.fit(xx.values.reshape(-1, 1),yy.values.reshape(-1, 1))
        r0 = -regresion_lineal.coef_
        lista_R0.append(r0)
    len(lista_R0)
    our_data_filter_un_final["r0"] = lista_R0
    return our_data_filter_un_final

def limpiezaPost(dt):
    our_data_filter_un_final = dt
    our_data_filter_un_final['r0'] = our_data_filter_un_final['r0'].str.get(0)
    our_data_filter_un_final['r0'] = our_data_filter_un_final['r0'].str.get(0)
    r0_puro = our_data_filter_un_final["r0"]
    our_data_filter_un_final["r0"] = r0_puro
    return our_data_filter_un_final

def saveParcial(dt):
    #Dejar los generados archivos en la misma carpeta
    dt.to_csv ('test.csv', mode='a', index = False, header=False)
    return 
#***************************************Este es el proceso principal************************************************************
#No es necesario hacer un Main pero si especificar cual es la funcion principal para agregarla en el servidor
# La idea es llamar a la funcion R0PanamaPorPartes() y que cumpla su funcion en el proceso general
def R0PanamaPorPartes():
    columnas = "CD_Corr,Corregimiento,Fecha,NuevosConfirmados,NuevosFallecidos,NuevosRecuperados,Poblacion,NuevosConfirmadosacum,NuevosRecuperadosacum,Susceptibles,pob_log_Susceptibles,r0"
    f = open ('test.csv','wb')
    f.write(columnas.encode())
    f.close()
    data = recuperarData()
    for corregimiento in data["Corregimiento"].unique():
        try:
            saveParcial(limpiezaPost(calculo(preCalculo(data[data["Corregimiento"] == corregimiento]))))
        except:
            print(corregimiento)
    return